# Tarefa 1 - Preparação do Dataset do Hippocampus

In [104]:
! pip install researchpy

In [105]:
import os
import pandas as pd
import sklearn as np
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [106]:
df_teste = pd.read_csv('./datasets/test_radiomics_hipocamp.csv')
df_treino = pd.read_csv('./datasets/train_radiomics_hipocamp.csv')
pd.options.display.max_columns = None

In [107]:
df_teste.shape

(100, 2180)

In [108]:
df_treino.shape

(305, 2181)

In [109]:
df_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 2180 entries, ID to Age
dtypes: float64(2011), int64(150), object(19)
memory usage: 1.7+ MB


In [110]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 2181 entries, ID to Transition
dtypes: float64(2014), int64(147), object(20)
memory usage: 5.1+ MB


### Colunas de valor único

Quando uma coluna contém o mesmo valor para todas as 305/100 entradas, esta não ajuda a distinguir entre as observações e não contribui para a construção de modelos preditivos, análises estatísticas, ou para entender padrões nos dados. Estas colunas podem ser consideradas redundantes e ocupam espaço desnecessário, o que também pode prejudicar a eficiência computacional ao aumentar o tempo de processamento. Com isto dito, em baixo pesquisamos todas essas colunas e eliminamos essas mesmo:

In [111]:
constant_columns_teste = df_teste.columns[df_teste.nunique() == 1]
print(constant_columns_teste)

Index(['diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy',
       'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet',
       'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings',
       'diagnostics_Configuration_EnabledImageTypes',
       'diagnostics_Image-original_Dimensionality',
       'diagnostics_Image-original_Spacing', 'diagnostics_Image-original_Size',
       ...
       'lbp-3D-m2_glszm_HighGrayLevelZoneEmphasis',
       'lbp-3D-m2_glszm_LowGrayLevelZoneEmphasis',
       'lbp-3D-m2_glszm_SizeZoneNonUniformity',
       'lbp-3D-m2_glszm_SizeZoneNonUniformityNormalized',
       'lbp-3D-m2_glszm_ZoneEntropy', 'lbp-3D-m2_ngtdm_Busyness',
       'lbp-3D-m2_ngtdm_Coarseness', 'lbp-3D-m2_ngtdm_Complexity',
       'lbp-3D-m2_ngtdm_Contrast', 'lbp-3D-m2_ngtdm_Strength'],
      dtype='object', length=159)


In [112]:
constant_columns_treino = df_treino.columns[df_treino.nunique() == 1]
print(constant_columns_treino)

Index(['diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy',
       'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet',
       'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings',
       'diagnostics_Configuration_EnabledImageTypes',
       'diagnostics_Image-original_Dimensionality',
       'diagnostics_Image-original_Spacing', 'diagnostics_Image-original_Size',
       ...
       'lbp-3D-m2_glszm_HighGrayLevelZoneEmphasis',
       'lbp-3D-m2_glszm_LowGrayLevelZoneEmphasis',
       'lbp-3D-m2_glszm_SizeZoneNonUniformity',
       'lbp-3D-m2_glszm_SizeZoneNonUniformityNormalized',
       'lbp-3D-m2_glszm_ZoneEntropy', 'lbp-3D-m2_ngtdm_Busyness',
       'lbp-3D-m2_ngtdm_Coarseness', 'lbp-3D-m2_ngtdm_Complexity',
       'lbp-3D-m2_ngtdm_Contrast', 'lbp-3D-m2_ngtdm_Strength'],
      dtype='object', length=159)


In [113]:
df_teste = df_teste.drop(constant_columns_teste, axis=1)
df_treino = df_treino.drop(constant_columns_treino, axis=1)

In [114]:
df_teste.shape

(100, 2021)

In [115]:
df_treino.shape

(305, 2022)

In [116]:
df_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 2021 entries, ID to Age
dtypes: float64(1991), int64(22), object(8)
memory usage: 1.5+ MB


In [117]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 2022 entries, ID to Transition
dtypes: float64(1994), int64(19), object(9)
memory usage: 4.7+ MB


Depois de feita a eliminação das colunas redundantes, através dos comando ***shape***, verificamos que eliminamos ***159 colunas redundantes*** dos dois datasets.
Com o comando ***info()***, verificamos para o dataset de teste, que dessas 159 colunas, 20 eram do tipo ***float64***, 128 eram do tipo ***int64*** e 11 eram do tipo ***object***.
Com o comando ***info()***, verificamos para o dataset de treino, que dessas 159 colunas, 20 eram do tipo ***float64***, 128 eram do tipo ***int64*** e 11 eram do tipo ***object***.

### Colunas duplicadas

Com as análises feitas aos datasets, descobrimos sobre a existência de colunas duplicadas, por isso o nosso próximo passo seria lidar com as mesmas porque podem causar problemas na previsão da coluna ***Transition***. 

In [118]:
def remove_duplicated_columns(df_teste):
    # Transpõe o DataFrame para identificar duplicações de colunas com base no conteúdo
    duplicated_columns = df_teste.T.duplicated(keep=False)
    
    # Filtra apenas as colunas duplicadas e seus conteúdos
    duplicated_data = df_teste.loc[:, duplicated_columns]
    
    # Agrupa as colunas duplicadas com base no conteúdo
    grouped_duplicates = duplicated_data.T.groupby(list(duplicated_data.T)).groups

    columns_to_remove = []
    # Loop por cada grupo de colunas duplicadas, mantendo apenas a primeira e removendo as demais
    for content, columns in grouped_duplicates.items():
        columns_to_remove.extend(columns[1:])  # Mantém a primeira coluna e remove as demais

    # Remove as colunas duplicadas do dataset
    df_teste.drop(columns=columns_to_remove, inplace=True)

    # Contagem total de colunas duplicadas removidas
    total_duplicated_columns = len(columns_to_remove)
    
    print(f"Total de colunas duplicadas removidas: {total_duplicated_columns}")
    
    # Retorna o DataFrame modificado e a contagem de colunas duplicadas
    return df_teste, total_duplicated_columns

# Uso da função
df_teste, total_duplicated_columns = remove_duplicated_columns(df_teste)

Total de colunas duplicadas removidas: 116


In [119]:
# Verificar novamente se há colunas duplicadas
duplicated_columns3 = df_teste.T.duplicated(keep=False)

# Exibir resultado
if duplicated_columns3.any():
    print("Ainda existem colunas duplicadas:")
    duplicate_column_names = df_teste.columns[duplicated_columns3]
    for col in duplicate_column_names:
        print(f"- {col}")
else:
    print("Não há mais colunas duplicadas.")

Não há mais colunas duplicadas.


In [120]:
df_teste.shape

(100, 1905)

In [121]:
df_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 1905 entries, ID to Age
dtypes: float64(1883), int64(15), object(7)
memory usage: 1.5+ MB


Depois de feita a eliminação das colunas duplicadas do dataset de teste, exceto uma, através dos comando ***shape***, verificamos que eliminamos ***116 colunas***.
Com o comando ***info()***, verificamos que dessas 116 colunas, 108 eram do tipo ***float64***, 7 eram do tipo ***int64*** e 1 era do tipo ***object***.

A seguir vamos efetuar o mesmo processo para o dataset de treino.

In [122]:
def remove_duplicated_columns(df_treino):
    # Transpõe o DataFrame para identificar duplicações de colunas com base no conteúdo
    duplicated_columns = df_treino.T.duplicated(keep=False)
    
    # Filtra apenas as colunas duplicadas e seus conteúdos
    duplicated_data = df_treino.loc[:, duplicated_columns]
    
    # Agrupa as colunas duplicadas com base no conteúdo
    grouped_duplicates = duplicated_data.T.groupby(list(duplicated_data.T)).groups

    columns_to_remove = []
    # Loop por cada grupo de colunas duplicadas, mantendo apenas a primeira e removendo as demais
    for content, columns in grouped_duplicates.items():
        columns_to_remove.extend(columns[1:])  # Mantém a primeira coluna e remove as demais

    # Remove as colunas duplicadas do dataset
    df_treino.drop(columns=columns_to_remove, inplace=True)

    # Contagem total de colunas duplicadas removidas
    total_duplicated_columns = len(columns_to_remove)
    
    print(f"Total de colunas duplicadas removidas: {total_duplicated_columns}")
    
    # Retorna o DataFrame modificado e a contagem de colunas duplicadas
    return df_treino, total_duplicated_columns

# Uso da função
df_treino, total_duplicated_columns = remove_duplicated_columns(df_treino)

Total de colunas duplicadas removidas: 113


In [123]:
# Verificar novamente se há colunas duplicadas
duplicated_columns3 = df_treino.T.duplicated(keep=False)

# Exibir resultado
if duplicated_columns3.any():
    print("Ainda existem colunas duplicadas:")
    duplicate_column_names = df_treino.columns[duplicated_columns3]
    for col in duplicate_column_names:
        print(f"- {col}")
else:
    print("Não há mais colunas duplicadas.")

Não há mais colunas duplicadas.


In [124]:
df_treino.shape

(305, 1909)

In [125]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 1909 entries, ID to Transition
dtypes: float64(1889), int64(12), object(8)
memory usage: 4.4+ MB


Depois de feita a eliminação das colunas duplicadas do dataset de treino, exceto uma, através dos comando ***shape***, verificamos que eliminamos ***113 colunas***.
Com o comando ***info()***, verificamos que dessas 113 colunas, 105 eram do tipo ***float64***, 7 eram do tipo ***int64*** e 1 era do tipo ***object***.

### Análise de colunas do tipo ***object***

Consideramos que as seguintes colunas a eliminar, não apresentam dados necessários para a previsão do atributo-objetivo, ***Transition:***
- ***ID:*** porque representa apenas o ID das imagens obtidas.
- ***Image e Mask:*** porque representa apenas a localização dos diferentes scans.
- ***diagnostics_Image-original_Hash:*** porque representa um hash único para a imagem original, usado para verificar a integridade da imagem ou identificar duplicates, mas para análises quantitativas não é importante.
- ***diagnostics_Mask-original_Hash:*** porque representa um hash único da mask original, utilizado para verificar a integridade da mask. Para análises quantitativas não é importante.

In [126]:
object_columns_teste = df_teste.select_dtypes(include='object')
print(object_columns_teste.head())

object_columns_treino = df_treino.select_dtypes(include='object')
print(object_columns_treino.head())

           ID                                              Image  \
0  941_S_1194  /notebooks/disk2/DS2_FreeSurfer/ADNI_941_S_119...   
1  036_S_0945  /notebooks/disk2/DS2_FreeSurfer/ADNI_036_S_094...   
2  024_S_1171  /notebooks/disk2/DS2_FreeSurfer/ADNI_024_S_117...   
3  035_S_0555  /notebooks/disk2/DS2_FreeSurfer/ADNI_035_S_055...   
4  023_S_0081  /notebooks/disk2/DS2_FreeSurfer/ADNI_023_S_008...   

                                                Mask  \
0  /notebooks/disk2/DS2_FreeSurfer/ADNI_941_S_119...   
1  /notebooks/disk2/DS2_FreeSurfer/ADNI_036_S_094...   
2  /notebooks/disk2/DS2_FreeSurfer/ADNI_024_S_117...   
3  /notebooks/disk2/DS2_FreeSurfer/ADNI_035_S_055...   
4  /notebooks/disk2/DS2_FreeSurfer/ADNI_023_S_008...   

            diagnostics_Image-original_Hash  \
0  b4977b66eca7c1e38c03fb5193cca7ca01dd46ec   
1  43850cb7e611c0438dbf2fee1f5800d548de621f   
2  a27b647efa22a9d77a17a07e92657722a268552d   
3  cbd443cf571aaa826bebc6cc9d4bf80f91afd938   
4  41ce7f75ad8398dc

In [127]:
df_teste = df_teste.drop('ID', axis=1)
df_teste = df_teste.drop('Image', axis=1)
df_teste = df_teste.drop('Mask', axis=1)
df_teste = df_teste.drop('diagnostics_Image-original_Hash', axis=1)
df_teste = df_teste.drop('diagnostics_Mask-original_Hash', axis=1)

df_treino = df_treino.drop('ID', axis=1)
df_treino = df_treino.drop('Image', axis=1)
df_treino = df_treino.drop('Mask', axis=1)
df_treino = df_treino.drop('diagnostics_Image-original_Hash', axis=1)
df_treino = df_treino.drop('diagnostics_Mask-original_Hash', axis=1)

In [128]:
df_teste.shape

(100, 1900)

In [129]:
df_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 1900 entries, diagnostics_Image-original_Mean to Age
dtypes: float64(1883), int64(15), object(2)
memory usage: 1.4+ MB


In [130]:
df_treino.shape

(305, 1904)

In [131]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 1904 entries, diagnostics_Image-original_Mean to Transition
dtypes: float64(1889), int64(12), object(3)
memory usage: 4.4+ MB


De seguida, vamos analisar as colunas restantes do tipo ***object***.

In [132]:
object_columns_teste = df_teste.select_dtypes(include='object')
print(object_columns_teste.head())

object_columns_treino = df_treino.select_dtypes(include='object')
print(object_columns_treino.head())

  diagnostics_Mask-original_BoundingBox  \
0             (92, 123, 90, 42, 29, 76)   
1             (79, 135, 90, 48, 16, 78)   
2            (105, 112, 87, 45, 27, 80)   
3             (88, 110, 88, 42, 30, 84)   
4             (88, 145, 94, 45, 21, 71)   

         diagnostics_Mask-original_CenterOfMassIndex  
0  (114.35013341239252, 137.93240438778537, 127.3...  
1  (105.5636763627475, 144.51344414568564, 127.57...  
2  (128.06696716033483, 125.64842240824211, 125.5...  
3  (110.5198398681267, 125.62310137760508, 130.65...  
4  (112.50337605070966, 155.2838638555877, 129.34...  
  diagnostics_Mask-original_BoundingBox  \
0            (103, 113, 93, 36, 30, 71)   
1             (81, 127, 93, 47, 16, 73)   
2             (77, 119, 89, 49, 30, 81)   
3             (93, 102, 90, 41, 29, 78)   
4             (87, 119, 91, 40, 27, 75)   

         diagnostics_Mask-original_CenterOfMassIndex Transition  
0  (121.94230227976358, 129.27272727272728, 128.4...      CN-CN  
1  (107.061704589278

Sobraram estas colunas. Reparamos que existiam duas colunas que continham dados em formato do tuplo. Logo, em vez de eliminar estes dados vamos abrir o tuplo e cada um dos elementos do tuplo, vai ter uma coluna designada e depois mais à frente vai se verificar se contêm dados importantes para a previsão.  

In [133]:
# Separar os elementos dos tuplos em colunas individuais
bbox_cols = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'bbox_x3', 'bbox_y3']
com_cols = ['com_x', 'com_y', 'com_z']

df_teste[bbox_cols] = pd.DataFrame(df_teste['diagnostics_Mask-original_BoundingBox'].apply(eval).tolist(), index=df_teste.index)
df_teste[com_cols] = pd.DataFrame(df_teste['diagnostics_Mask-original_CenterOfMassIndex'].apply(eval).tolist(), index=df_teste.index)

# Remover as colunas originais
df_teste = df_teste.drop('diagnostics_Mask-original_BoundingBox', axis = 1)
df_teste = df_teste.drop('diagnostics_Mask-original_CenterOfMassIndex', axis=1)

# Explicação das colunas
# diagnostics_Mask-original_BoundingBox: Contém as coordenadas dos vértices da caixa delimitadora (bounding box).
# diagnostics_Mask-original_CenterOfMassIndex: Contém as coordenadas do centro de massa.

# Verificar a existência de valores ausentes
missing_values = df_teste.isnull().sum()

if missing_values.any():
    print('Sim')
    
    # Imprimir os nomes das colunas com valores ausentes
    cols_with_missing_values = missing_values[missing_values > 0].index.tolist()
    print("Colunas com valores ausentes:", cols_with_missing_values)
else:
    print('Não')

Não


In [134]:
# Imprimir as primeiras linhas das colunas especificadas
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'bbox_x3', 'bbox_y3', 'com_x', 'com_y', 'com_z']
print(df_teste[columns].head())

   bbox_x1  bbox_y1  bbox_x2  bbox_y2  bbox_x3  bbox_y3       com_x  \
0       92      123       90       42       29       76  114.350133   
1       79      135       90       48       16       78  105.563676   
2      105      112       87       45       27       80  128.066967   
3       88      110       88       42       30       84  110.519840   
4       88      145       94       45       21       71  112.503376   

        com_y       com_z  
0  137.932404  127.331456  
1  144.513444  127.574921  
2  125.648422  125.565615  
3  125.623101  130.657365  
4  155.283864  129.348767  


In [135]:
df_teste.shape

(100, 1907)

In [136]:
df_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 1907 entries, diagnostics_Image-original_Mean to com_z
dtypes: float64(1886), int64(21)
memory usage: 1.5 MB


In [137]:
# Separar os elementos dos tuplos em colunas individuais
bbox_cols = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'bbox_x3', 'bbox_y3']
com_cols = ['com_x', 'com_y', 'com_z']

df_treino[bbox_cols] = pd.DataFrame(df_treino['diagnostics_Mask-original_BoundingBox'].apply(eval).tolist(), index=df_treino.index)
df_treino[com_cols] = pd.DataFrame(df_treino['diagnostics_Mask-original_CenterOfMassIndex'].apply(eval).tolist(), index=df_treino.index)

# Remover as colunas originais
df_treino = df_treino.drop('diagnostics_Mask-original_BoundingBox', axis = 1)
df_treino = df_treino.drop('diagnostics_Mask-original_CenterOfMassIndex', axis=1)

# Explicação das colunas
# diagnostics_Mask-original_BoundingBox: Contém as coordenadas dos vértices da caixa delimitadora (bounding box).
# diagnostics_Mask-original_CenterOfMassIndex: Contém as coordenadas do centro de massa.

# Verificar a existência de valores ausentes
missing_values = df_treino.isnull().sum()

if missing_values.any():
    print('Sim')
    
    # Imprimir os nomes das colunas com valores ausentes
    cols_with_missing_values = missing_values[missing_values > 0].index.tolist()
    print("Colunas com valores ausentes:", cols_with_missing_values)
else:
    print('Não')

Não


In [138]:
# Imprimir as primeiras linhas das colunas especificadas
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'bbox_x3', 'bbox_y3', 'com_x', 'com_y', 'com_z']
print(df_treino[columns].head())

   bbox_x1  bbox_y1  bbox_x2  bbox_y2  bbox_x3  bbox_y3       com_x  \
0      103      113       93       36       30       71  121.942302   
1       81      127       93       47       16       73  107.061705   
2       77      119       89       49       30       81  103.364097   
3       93      102       90       41       29       78  116.298273   
4       87      119       91       40       27       75  108.265620   

        com_y       com_z  
0  129.272727  128.404025  
1  135.280884  128.274585  
2  135.281646  128.986283  
3  118.674315  129.309866  
4  132.054627  127.672068  


In [139]:
df_treino.shape

(305, 1911)

In [140]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 1911 entries, diagnostics_Image-original_Mean to com_z
dtypes: float64(1892), int64(18), object(1)
memory usage: 4.4+ MB


Eliminamos as seguintes colunas, ***diagnostics_Mask-original_BoundingBox', diagnostics_Mask-original_CenterOfMassIndex***, e criamos novas com cada um dos elementos de coordenadas dos tuplos ***['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'bbox_x3', 'bbox_y3', 'com_x', 'com_y', 'com_z'].***

### Tratamento de Outliers

Começamos por tratar os outliers do dataset de teste.

In [141]:
# Copiando o dataframe original para preservar dados antes da transformação
df_teste_tratado = df_teste.copy()

# Loop pelas colunas para calcular IQR e ajustar os outliers
for coluna in df_teste_tratado.select_dtypes(include=['float', 'int']).columns:  # Apenas colunas numéricas
    # Calcula Q1 e Q3
    Q1 = df_teste_tratado[coluna].quantile(0.25)
    Q3 = df_teste_tratado[coluna].quantile(0.75)
    IQR = Q3 - Q1
    
    # Limites para detecção de outliers
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    # Tratar os outliers ajustando-os aos limites
    df_teste_tratado[coluna] = df_teste_tratado[coluna].apply(lambda x: limite_inferior if x < limite_inferior else (limite_superior if x > limite_superior else x))

# Verificação dos ajustes
print("Outliers tratados!")
df_teste_tratado.describe()

Outliers tratados!


diagnostics_Image-original_Mean  diagnostics_Image-original_Maximum  \
count                       100.000000                          100.000000   
mean                          5.768584                          168.605000   
std                           0.699911                           23.824476   
min                           4.108809                          126.000000   
25%                           5.275968                          153.750000   
50%                           5.670718                          168.000000   
75%                           6.191261                          180.500000   
max                           7.564201                          220.625000   

       diagnostics_Mask-original_VoxelNum  original_shape_Elongation  \
count                          100.000000                 100.000000   
mean                          6708.457500                   0.419020   
std                           1103.021617                   0.031074   
min                           4011.000000                   0.342218   
25%                           6039.750000                   0.399488   
50%                           6588.500000                   0.419641   
75%                           7487.000000                   0.437669   
max                           9657.875000                   0.494939   

       original_shape_Flatness  original_shape_LeastAxisLength  \
count               100.000000                      100.000000   
mean                  0.097937                       10.149291   
std                   0.008831                        0.836959   
min                   0.080628                        8.329650   
25%                   0.091993                        9.552989   
50%                   0.096567                       10.093060   
75%                   0.103951                       10.606888   
max                   0.121887                       12.187736   

       original_shape_MajorAxisLength  original_shape_Maximum2DDiameterColumn  \
count                      100.000000                              100.000000   
mean                       103.699506                               74.196031   
std                          5.643417                                3.862953   
min                         90.895819                               65.802736   
25%                         99.808994                               71.918073   
50%                        103.624223                               73.549303   
75%                        106.550650                               76.512901   
max                        116.663133                               83.405144   

       original_shape_Maximum2DDiameterRow  \
count                           100.000000   
mean                             74.278766   
std                               3.774507   
min                              66.068147   
25%                              72.006944   
50%                              74.060786   
75%                              76.138011   
max                              82.334612   

       original_shape_Maximum2DDiameterSlice  \
count                             100.000000   
mean                               43.456956   
std                                 2.815202   
min                                35.500828   
25%                                41.566186   
50%                                43.416587   
75%                                45.609757   
max                                50.447993   

       original_shape_Maximum3DDiameter  original_shape_MeshVolume  \
count                        100.000000                 100.000000   
mean                          75.424702                6643.033958   
std                            3.868985                1097.317369   
min                           66.439063                3948.000000   
25%                           73.331353                5995.302083   
50%                           74.655855                6541.04

Em seguida, vamos tratar dos outliers do dataset de treino.

In [142]:
# Copiando o dataframe original para preservar dados antes da transformação
df_treino_tratado = df_treino.copy()

# Loop pelas colunas para calcular IQR e ajustar os outliers
for coluna in df_treino_tratado.select_dtypes(include=['float', 'int']).columns:  # Apenas colunas numéricas
    # Calcula Q1 e Q3
    Q1 = df_treino_tratado[coluna].quantile(0.25)
    Q3 = df_treino_tratado[coluna].quantile(0.75)
    IQR = Q3 - Q1
    
    # Limites para detecção de outliers
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    # Tratar os outliers ajustando-os aos limites
    df_treino_tratado[coluna] = df_treino_tratado[coluna].apply(lambda x: limite_inferior if x < limite_inferior else (limite_superior if x > limite_superior else x))

# Verificação dos ajustes
print("Outliers tratados!")
df_treino_tratado.describe()


Outliers tratados!


diagnostics_Image-original_Mean  diagnostics_Image-original_Maximum  \
count                       305.000000                          305.000000   
mean                          5.810469                          173.378689   
std                           0.613858                           23.722145   
min                           4.321602                          128.000000   
25%                           5.317298                          157.000000   
50%                           5.811449                          172.000000   
75%                           6.237751                          190.000000   
max                           7.583210                          239.500000   

       diagnostics_Mask-original_VoxelNum  original_shape_Elongation  \
count                          305.000000                 305.000000   
mean                          6715.280328                   0.411826   
std                           1065.417404                   0.028940   
min                           3758.500000                   0.334928   
25%                           5941.000000                   0.392944   
50%                           6723.000000                   0.412479   
75%                           7396.000000                   0.431621   
max                           9453.000000                   0.489636   

       original_shape_Flatness  original_shape_LeastAxisLength  \
count               305.000000                      305.000000   
mean                  0.096690                       10.113897   
std                   0.008926                        0.831477   
min                   0.075142                        8.420713   
25%                   0.089977                        9.525253   
50%                   0.096609                       10.106838   
75%                   0.102240                       10.635093   
max                   0.120634                       12.299853   

       original_shape_MajorAxisLength  original_shape_Maximum2DDiameterColumn  \
count                      305.000000                              305.000000   
mean                       104.855648                               74.599399   
std                          6.098181                                4.043955   
min                         90.675836                               64.327288   
25%                        100.501871                               71.568149   
50%                        104.370667                               74.545288   
75%                        108.677326                               77.317527   
max                        120.940509                               85.941594   

       original_shape_Maximum2DDiameterRow  \
count                           305.000000   
mean                             74.625719   
std                               4.009993   
min                              65.069194   
25%                              71.568149   
50%                              75.006666   
75%                              77.233412   
max                              85.731307   

       original_shape_Maximum2DDiameterSlice  \
count                             305.000000   
mean                               43.088716   
std                                 2.447864   
min                                36.674242   
25%                                41.436699   
50%                                43.081318   
75%                                44.721360   
max                                48.662100   

       original_shape_Maximum3DDiameter  original_shape_MeshVolume  \
count                        305.000000                 305.000000   
mean                          75.911589                6652.143443   
std                            3.960366                1059.858063   
min                           66.610810                3654.833333   
25%                           73.116346                5876.333333   
50%                           76.124897                6640.50

### Conversão de int64 e float64 para int32 e float32 

Começamos por tratar das conversões dos ints e floats do dataset de teste.

In [143]:
float_features = df_teste_tratado.select_dtypes(include='float')
int_features = df_teste_tratado.select_dtypes(include='int')

df_teste_tratado[float_features.columns] = df_teste_tratado[float_features.columns].astype(np.float32)
df_teste_tratado[int_features.columns] = df_teste_tratado[int_features.columns].astype(np.int32)
df_teste_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 1907 entries, diagnostics_Image-original_Mean to com_z
dtypes: float32(1898), int32(9)
memory usage: 745.1 KB


Em seguida, vamos tratar das conversões dos ints e floats do dataset de treino.

In [144]:
float_features = df_treino_tratado.select_dtypes(include='float')
int_features = df_treino_tratado.select_dtypes(include='int')

df_treino_tratado[float_features.columns] = df_treino_tratado[float_features.columns].astype(np.float32)
df_treino_tratado[int_features.columns] = df_treino_tratado[int_features.columns].astype(np.int32)
df_treino_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 1911 entries, diagnostics_Image-original_Mean to com_z
dtypes: float32(1904), int32(6), object(1)
memory usage: 2.2+ MB


### Exportar os datasets resultantes

In [145]:
# Criar a pasta se ela não existir
output_folder = 'data_mod_hippo'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Exportar o DataFrame para um arquivo CSV na pasta criada
output_file = os.path.join(output_folder, 'dataset_teste.csv')
df_teste_tratado.to_csv(output_file, index=False)

print(f"Dataset final de teste limpo exportado para {output_file}")

Dataset final de teste limpo exportado para data_mod_hippo/dataset_teste.csv


In [146]:
# Criar a pasta se ela não existir
output_folder = 'data_mod_hippo'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Exportar o DataFrame para um arquivo CSV na pasta criada
output_file = os.path.join(output_folder, 'dataset_treino.csv')
df_treino_tratado.to_csv(output_file, index=False)

print(f"Dataset final de treino limpo exportado para {output_file}")

Dataset final de treino limpo exportado para data_mod_hippo/dataset_treino.csv
